#Clean and Filter

In [ ]:
import pandas as pd
data = pd.read_csv("/content/all_articles_More.csv")

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string 
from nltk.corpus import stopwords

stop = stopwords.words('english')

#convert tweets to lower case
data['preprocess_data'] = data['Sent'].str.lower()

#url removes
data['preprocess_data'] = data['preprocess_data'].str.replace(r'(https|http)?:\/(\w|\.|\/|\?|\=|\&|\%)*\b','')
data['preprocess_data'] = data['preprocess_data'].str.replace(r'www\.\S+\.com','')

#removes retweets & cc
data['preprocess_data'] = data['preprocess_data'].str.replace(r'rt|cc', '')

#hashtags removes
data['preprocess_data'] = data['preprocess_data'].str.replace(r'#\S+', '')

#user mention removes
data['preprocess_data'] = data['preprocess_data'].str.replace(r'@\S+', '')

#emoji 
data['preprocess_data'] = data['preprocess_data'].str.replace(r'[^\x00-\x7F]+', '')

#html tags
data['preprocess_data'] = data['preprocess_data'].str.replace(r'<.*?>', '')

#removes extra spaces
data['preprocess_data'] = data['preprocess_data'].str.replace(r' +', ' ')

#punctuation
data['preprocess_data'] = data['preprocess_data'].str.replace('[{}]'.format(string.punctuation), '')

#stop words removes
data['preprocess_data'] = data['preprocess_data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


#convert preprocessed list words to string 
data['preprocess_str'] = data['preprocess_data'].apply(' '.join)

data

,Sent,Len,label,preprocess_data
0,['Preeclampsia (PE) is a hypertensive disorder of pregnancy that can cause detrimental obstetric outcomes if not managed properly.,17,Coronary artery disease,['preeclampsia (pe) hypertensive disorder pregnancy cause detrimental obstetric outcomes managed properly.
1,Current evidence demonstrates higher risk for long-term cardiovascular disease in preeclamptic women.,10,Coronary artery disease,current evidence demonstrates higher risk long-term cardiovascular disease preeclamptic women.
2,"Even in uncomplicated pregnancies, the heart work overload often reveals subtle cardiac defects or abnormalities, which otherwise remain undiagnosed in women without a history of pregnancy.",24,Coronary artery disease,"even uncomplicated pregnancies, heart work overload often reveals subtle cardiac defects abnormalities, otherwise remain undiagnosed women without history pregnancy."
3,Pathophysiologic patterns occurring in PE patients resemble biochemical responses observed in cases of cardiovascular disease.,13,Coronary artery disease,pathophysiologic patterns occurring pe patients resemble biochemical responses observed cases cardiovascular disease.
4,It has been estimated that women with an obstetric history of PE are more likely to develop coronary artery disease in the long run.,23,Coronary artery disease,estimated women obstetric history pe likely develop coronary artery disease long run.
...,...,...,...,...
17620,"['OBJECTIVES: Pizza eating has been favourably related to the risk of cardiovascular disease, but the data are limited.",17,unsaturated fats disease,"['objectives: pizza eating favourably related risk cardiovascular disease, data limited."
17621,"To evaluate the potential role of pizza consumption on the risk of acute myocardial infarction (AMI), we considered data from an Italian study. DESIGN: We conducted a hospital-based case-control study on 507 cases of nonfatal AMI and 478 controls in Milan, Italy, between 1995 and 1999. RESULTS: The multivariate odds ratios were 0.78 for occasional, 0.62 for regular and 0.44 for frequent eaters.",59,unsaturated fats disease,"evaluate potential role pizza consumption risk acute myocardial infarction (ami), considered data italian study. design: conducted hospital-based case-control study 507 cases nonfatal ami 478 controls milan, italy, 1995 1999. results: multivariate odds ratios 0.78 occasional, 0.62 regular 0.44 frequent eaters."
17622,"The estimates were similar across strata of age, sex, smoking and other major covariates. CONCLUSIONS: Some of the ingredients of pizza have been shown to have a favourable influence on the risk of cardiovascular disease.",32,unsaturated fats disease,"estimates similar across strata age, sex, smoking major covariates. conclusions: ingredients pizza shown favourable influence risk cardiovascular disease."
17623,"However, there is no single explanation for the present findings.",10,unsaturated fats disease,"however, single explanation present findings."


In [ ]:
#https://shubham-agnihotri.medium.com/sentimental-analysis-using-rule-based-approach-text-blob-over-twitter-data-part-i-391edb24aa58
from textblob import TextBlob
from tweepy import OAuthHandler, Stream, Cursor, API
from tweepy.streaming import StreamListener
import tweepy
import pandas as pd
import numpy as np
import json
import time
import re
import os.path

class Analyze_Tweet():

    def Analyze_Sentiment(self,tweet):
        try:
            analysis_result = TextBlob(tweet)
            if analysis_result.sentiment.polarity > 0:
                return "Positive"
            elif analysis_result.sentiment.polarity == 0:
                return "Neutral"
            else:
                return "Negative"
                
        except Exception as e:
            print("Exception as Analyze_Tweet Class on function Analyze_Sentiment: ",e)
            return "Nil"

    def Analyze_Sentiment_value(self,tweet):
        try:
            analysis_result = TextBlob(tweet)
            return analysis_result.sentiment.polarity 
               
        except Exception as e:
            print("Exception as Analyze_Tweet Class on function Analyze_Sentiment: ",e)
            return "Nil"

In [ ]:
class Analyze_Tweet2():

    def Analyze_subjectivity(self,tweet):
        try:
            analysis_result = TextBlob(tweet)
            if analysis_result.sentiment.subjectivity > 0.5:
                return "Subjective"
            elif analysis_result.sentiment.subjectivity == 0.5:
                return "Neutral"
            else:
                return "Objective"
                
        except Exception as e:
            print("Exception as Analyze_Tweet2 Class on function Analyze_subjectivity: ",e)
            return "Nil"

    def Analyze_subjectivity_value(self,tweet):
        try:
            analysis_result = TextBlob(tweet)
            return analysis_result.sentiment.subjectivity
               
        except Exception as e:
            print("Exception as Analyze_Tweet Class on function Analyze_Sentiment: ",e)
            return "Nil"

In [ ]:
a = Analyze_Tweet()
data['sentiments'] = np.array([a.Analyze_Sentiment(row[3]) for index,row in data.iterrows()])
print(data.columns)
df2 = data.filter(['tweet','sentiments'],axis = 1)
print(df2)

Index(['Sent', 'Len', 'label', 'preprocess_data', 'sentiments'], dtype='object')
      sentiments
0        Neutral
1       Positive
2       Negative
3        Neutral
4       Negative
...          ...
17620   Negative
17621   Positive
17622   Positive
17623   Negative
17624   Positive

[17625 rows x 1 columns]


In [ ]:
a = Analyze_Tweet()
data['sentiments_val'] = np.array([a.Analyze_Sentiment_value(row[0]) for index,row in data.iterrows()])
print(data.columns)
df2 = data.filter(['tweet','sentiments_val'],axis = 1)
print(df2)

Index(['Sent', 'Len', 'label', 'preprocess_data', 'sentiments',
       'sentiments_val'],
      dtype='object')
       sentiments_val
0            0.000000
1            0.125000
2           -0.191667
3            0.000000
4            0.150000
...               ...
17620       -0.035714
17621        0.116667
17622       -0.020833
17623        0.017857
17624        0.016667

[17625 rows x 1 columns]


In [ ]:
data['sentiments'].value_counts()

Positive    8289
Neutral     5455
Negative    3881
Name: sentiments, dtype: int64

In [ ]:
a = Analyze_Tweet2()
data['subjective'] = np.array([a.Analyze_subjectivity(row[3]) for index,row in data.iterrows()])
print(data.columns)
df2 = data.filter(['tweet','subjective'],axis = 1)
print(df2)

Index(['Sent', 'Len', 'label', 'preprocess_data', 'sentiments',
       'sentiments_val', 'subjective'],
      dtype='object')
       subjective
0       Objective
1       Objective
2       Objective
3       Objective
4      Subjective
...           ...
17620   Objective
17621   Objective
17622   Objective
17623   Objective
17624     Neutral

[17625 rows x 1 columns]


In [ ]:
a = Analyze_Tweet2()
data['subjective_val'] = np.array([a.Analyze_subjectivity_value(row[0]) for index,row in data.iterrows()])
print(data.columns)
df2 = data.filter(['tweet','subjective_val'],axis = 1)
print(df2)

Index(['Sent', 'Len', 'label', 'preprocess_data', 'sentiments',
       'sentiments_val', 'subjective', 'subjective_val'],
      dtype='object')
       subjective_val
0            0.100000
1            0.450000
2            0.250000
3            0.000000
4            0.633333
...               ...
17620        0.271429
17621        0.400321
17622        0.425000
17623        0.107143
17624        0.500000

[17625 rows x 1 columns]


In [ ]:
data['subjective'].value_counts()

Objective     12010
Subjective     4582
Neutral        1033
Name: subjective, dtype: int64

**Vader**

In [ ]:
pip install vaderSentiment

In [ ]:
# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):

	# Create a SentimentIntensityAnalyzer object.
	sid_obj = SentimentIntensityAnalyzer()


	sentiment_dict = sid_obj.polarity_scores(sentence)
	
	# decide sentiment as positive, negative and neutral
	if sentiment_dict['compound'] >= 0.05 :
		return "Positive"#,sentiment_dict['compound'] *100

	elif sentiment_dict['compound'] <= - 0.05 :
		return "Negative"#,sentiment_dict['compound'] *100

	else :
		return "Neutral"#,sentiment_dict['compound'] *100


def sentiment_scores2(sentence):

	# Create a SentimentIntensityAnalyzer object.
	sid_obj = SentimentIntensityAnalyzer()

	# which contains pos, neg, neu, and compound scores.
	sentiment_dict = sid_obj.polarity_scores(sentence)

	return sentiment_dict['compound']#,sentiment_dict['compound'] *100



In [ ]:

data['vader_sentiments']= np.array([sentiment_scores(row[0]) for index,row in data.iterrows()])

data['vader_sentiments_val']= np.array([sentiment_scores2(row[0]) for index,row in data.iterrows()])

data


,Sent,Len,label,preprocess_data,sentiments,sentiments_val,subjective,subjective_val,vader_sentiments,vader_sentiments_val
0,['Preeclampsia (PE) is a hypertensive disorder of pregnancy that can cause detrimental obstetric outcomes if not managed properly.,17,Coronary artery disease,['preeclampsia (pe) hypertensive disorder pregnancy cause detrimental obstetric outcomes managed properly.,Neutral,0.000000,Objective,0.100000,Negative,-0.4019
1,Current evidence demonstrates higher risk for long-term cardiovascular disease in preeclamptic women.,10,Coronary artery disease,current evidence demonstrates higher risk long-term cardiovascular disease preeclamptic women.,Positive,0.125000,Objective,0.450000,Negative,-0.2732
2,"Even in uncomplicated pregnancies, the heart work overload often reveals subtle cardiac defects or abnormalities, which otherwise remain undiagnosed in women without a history of pregnancy.",24,Coronary artery disease,"even uncomplicated pregnancies, heart work overload often reveals subtle cardiac defects abnormalities, otherwise remain undiagnosed women without history pregnancy.",Negative,-0.191667,Objective,0.250000,Neutral,0.0000
3,Pathophysiologic patterns occurring in PE patients resemble biochemical responses observed in cases of cardiovascular disease.,13,Coronary artery disease,pathophysiologic patterns occurring pe patients resemble biochemical responses observed cases cardiovascular disease.,Neutral,0.000000,Objective,0.000000,Neutral,0.0000
4,It has been estimated that women with an obstetric history of PE are more likely to develop coronary artery disease in the long run.,23,Coronary artery disease,estimated women obstetric history pe likely develop coronary artery disease long run.,Negative,0.150000,Subjective,0.633333,Neutral,0.0000
...,...,...,...,...,...,...,...,...,...,...
17620,"['OBJECTIVES: Pizza eating has been favourably related to the risk of cardiovascular disease, but the data are limited.",17,unsaturated fats disease,"['objectives: pizza eating favourably related risk cardiovascular disease, data limited.",Negative,-0.035714,Objective,0.271429,Negative,-0.4404
17621,"To evaluate the potential role of pizza consumption on the risk of acute myocardial infarction (AMI), we considered data from an Italian study. DESIGN: We conducted a hospital-based case-control study on 507 cases of nonfatal AMI and 478 controls in Milan, Italy, between 1995 and 1999. RESULTS: The multivariate odds ratios were 0.78 for occasional, 0.62 for regular and 0.44 for frequent eaters.",59,unsaturated fats disease,"evaluate potential role pizza consumption risk acute myocardial infarction (ami), considered data italian study. design: conducted hospital-based case-control study 507 cases nonfatal ami 478 controls milan, italy, 1995 1999. results: multivariate odds ratios 0.78 occasional, 0.62 regular 0.44 frequent eaters.",Positive,0.116667,Objective,0.400321,Negative,-0.2732
17622,"The estimates were similar across strata of age, sex, smoking and other major covariates. CONCLUSIONS: Some of the ingredients of pizza have been shown to have a favourable influence on the risk of cardiovascular disease.",32,unsaturated fats disease,"estimates similar across strata age, sex, smoking major covariates. conclusions: ingredients pizza shown favourable influence risk cardiovascular disease.",Positive,-0.020833,Objective,0.425000,Negative,-0.2732
17623,"However, there is no single explanation for the present findings.",10,unsaturated fats disease,"however, single explanation present findings.",Negative,0.017857,Objective,0.107143,Negative,-0.2960


In [ ]:
data['vader_sentiments'].value_counts()

Positive    8213
Negative    5217
Neutral     4195
Name: vader_sentiments, dtype: int64

##Flair Context Filtration

In [ ]:
pip install flair

**Zero Shot Predictions**

using TARS to get if statement about disease or not



In [ ]:
from flair.models.text_classification_model import TARSClassifier
from flair.data import Sentence

# 1. Load our pre-trained TARS model for English
tars = TARSClassifier.load('tars-base')

classes = ['drug','symptoms','heart disease','disease','treatment']
flair_tars=[]
flair_tars_len=[]

for i in range(len(data['Sent'])):
    sentence = Sentence(data['Sent'][i])
    tars.predict_zero_shot(sentence, classes)
    print(i)
    flair_tars.append([sent for sent in sentence.labels])
    flair_tars_len.append(len([sent for sent in sentence.labels]))

data['flair_tars']=flair_tars
data['flair_tars_len']=flair_tars_len
data['flair_tars']

In [ ]:
pd.set_option('display.max_colwidth', None)
data[['flair_tars','Sent','label']]

,flair_tars,Sent,label
0,"[heart disease (0.6824), treatment (0.6469), symptoms (0.8362), disease (0.9684)]",['Preeclampsia (PE) is a hypertensive disorder of pregnancy that can cause detrimental obstetric outcomes if not managed properly.,Coronary artery disease
1,"[heart disease (0.9719), disease (0.9598)]",Current evidence demonstrates higher risk for long-term cardiovascular disease in preeclamptic women.,Coronary artery disease
2,"[heart disease (0.9775), symptoms (0.896), disease (0.9628)]","Even in uncomplicated pregnancies, the heart work overload often reveals subtle cardiac defects or abnormalities, which otherwise remain undiagnosed in women without a history of pregnancy.",Coronary artery disease
3,"[heart disease (0.965), treatment (0.9188), symptoms (0.7273), disease (0.9858), drug (0.8282)]",Pathophysiologic patterns occurring in PE patients resemble biochemical responses observed in cases of cardiovascular disease.,Coronary artery disease
4,"[heart disease (0.9899), treatment (0.694), symptoms (0.9561), disease (0.9822)]",It has been estimated that women with an obstetric history of PE are more likely to develop coronary artery disease in the long run.,Coronary artery disease
...,...,...,...
17620,"[heart disease (0.9741), treatment (0.951), symptoms (0.9365), disease (0.991), drug (0.916)]","['OBJECTIVES: Pizza eating has been favourably related to the risk of cardiovascular disease, but the data are limited.",unsaturated fats disease
17621,"[heart disease (0.9446), disease (0.9951), drug (0.8412)]","To evaluate the potential role of pizza consumption on the risk of acute myocardial infarction (AMI), we considered data from an Italian study. DESIGN: We conducted a hospital-based case-control study on 507 cases of nonfatal AMI and 478 controls in Milan, Italy, between 1995 and 1999. RESULTS: The multivariate odds ratios were 0.78 for occasional, 0.62 for regular and 0.44 for frequent eaters.",unsaturated fats disease
17622,[],"The estimates were similar across strata of age, sex, smoking and other major covariates. CONCLUSIONS: Some of the ingredients of pizza have been shown to have a favourable influence on the risk of cardiovascular disease.",unsaturated fats disease
17623,[],"However, there is no single explanation for the present findings.",unsaturated fats disease


In [ ]:
data[data['flair_tars_len']==0]